<a href="https://colab.research.google.com/github/forestmontagne/enfi2025/blob/main/classified2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap  # Optional, for interactive mapping

In [ ]:
ee.Authenticate()
ee.Initialize(project = 'ee-abdochem072', opt_url = 'https://earthengine-highvolume.googleapis.com')

In [ ]:
# Define constants
AOI = ee.FeatureCollection('projects/ee-abdochem071/assets/TalassemtaneWGS84')
START_DATE = '2020-01-01'
END_DATE = '2021-01-01'
CLOUD_COVER_THRESHOLD = 50
VIS_PARAMS = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.2}

In [ ]:
# Utility Functions
def apply_l8_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

In [ ]:
# Utility Functions
def apply_l8_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

def apply_l7_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_band = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_band, None, True)

def mask_sr_clouds(image):
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(31).eq(0)  # 31 = '11111' in binary
    saturation_mask = image.select('QA_RADSAT').eq(0)
    return image.updateMask(qa_mask).updateMask(saturation_mask)

def rename_l7_bands(image):
    return image.select(
        ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    )
def add_indices(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('ndvi')
    ndbi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndbi')
    mndwi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('mndwi')
    bsi = image.expression(
        '((swir1 + red) - (nir + blue)) / ((swir1 + red) + (nir + blue))',
        {
            'swir1': image.select('SR_B6'),
            'red': image.select('SR_B4'),
            'nir': image.select('SR_B5'),
            'blue': image.select('SR_B2')
        }
    ).rename('bsi')
    return image.addBands([ndvi, ndbi, mndwi, bsi])

def process_landsat(collection_id, scale_func, band_rename_func=None):
    collection = (ee.ImageCollection(collection_id)
                  .filterBounds(AOI)
                  .filterDate(START_DATE, END_DATE)
                  .filter(ee.Filter.lt('CLOUD_COVER', CLOUD_COVER_THRESHOLD))
                  .map(mask_sr_clouds)
                  .map(scale_func))

    if band_rename_func:
        collection = collection.map(band_rename_func)

    return collection.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

In [ ]:
# Process Landsat 8 and 7
l8_collection = process_landsat('LANDSAT/LC08/C02/T1_L2', apply_l8_scale_factors)
l7_collection = process_landsat('LANDSAT/LE07/C02/T1_L2', apply_l7_scale_factors, rename_l7_bands)

# Merge and create composite
landsat78 = l7_collection.merge(l8_collection).map(lambda img: img.toFloat())
composite = landsat78.median().clip(AOI)
composite_with_indices = add_indices(composite)

# Elevation and Slope
elevation_collection = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM')
elevation = elevation_collection.mean().clip(AOI)
slope = ee.Terrain.slope(
    elevation_collection.mosaic().setDefaultProjection(elevation_collection.first().projection())
)
final_composite = composite_with_indices.addBands([elevation.rename('elevation'), slope.rename('slope')])

In [ ]:
# Create a default map
Map = geemap.Map()


In [ ]:
landsat_vis = {
    'min': 0,
    'max': 0.2,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}
Map.addLayer(final_composite, landsat_vis, 'false color composite')

In [ ]:
# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Classification (Assuming forest and noforest are defined elsewhere)
# Replace with actual FeatureCollections if available
#forest = ee.FeatureCollection([])  # Placeholder
#noforest = ee.FeatureCollection([])  # Placeholder
gcps = ee.FeatureCollection('projects/ee-abdochem071/assets/training2020')
training_gcp = gcps.filter(ee.Filter.lt('random', 0.6))
validation_gcp = gcps.filter(ee.Filter.gte('random', 0.6))

training = final_composite.sampleRegions(
    collection=training_gcp,
    properties=['landcover'],
    scale=10,
    tileScale=4
)

classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='landcover',
    inputProperties=final_composite.bandNames()
)

classified = final_composite.classify(classifier)
Map.addLayer(classified.randomVisualizer(), {}, "clusters")
Map

#Map.addLayer(classified, {}, 'Classification');
#Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
legend_keys = ["One", "Two"]
legend_colors = ["green", "red"]

# Reclassify the map
classified = classified.remap([0, 1], [1, 2])

Map.addLayer(
    classified, {"min": 1, "max": 2, "palette": legend_colors}, "Labelled clusters"
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position="bottomright"
)
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
# Export function
def export_image(image, description, file_prefix, region):
    ee.batch.Export.image.toDrive(
        image=classified,
        description='classified2020',
        folder='earthengine',
        fileNamePrefix=file_prefix,
        region=region,
        scale=30,
        maxPixels=1e9
    ).start()

# Export images
export_image(classified, 'TNP_classified', 'TNP_classified', AOI)
export_image(composite.visualize(**VIS_PARAMS), '2020_Composite_Visualized', 'TNP_composite_visualized_2020', AOI)

In [ ]:
# Area Calculation
forest_area = (classified.eq(0)
               .multiply(ee.Image.pixelArea())
               .reduceRegion(
                   reducer=ee.Reducer.sum(),
                   geometry=AOI.geometry(),
                   scale=10,
                   tileScale=4,
                   maxPixels=1e10
               ).getNumber('classification')
               .divide(1e4)
               .round())

total_area = ee.Number(AOI.geometry().area()).divide(1e4).round()

print('Forest area (hectares):', forest_area.getInfo())
print('Total area (hectares):', total_area.getInfo())

EEException: Dictionary.getNumber: Dictionary does not contain key: 'classification'.

In [ ]:
# Accuracy Assessment
validation = final_composite.sampleRegions(
    collection=validation_gcp,
    properties=['landcover'],
    scale=10,
    tileScale=16
)

test = validation.classify(classifier)
confusion_matrix = test.errorMatrix('landcover', 'classification')

print('Confusion Matrix:', confusion_matrix.getInfo())
print('Overall Accuracy:', confusion_matrix.accuracy().getInfo())
print('Kappa:', confusion_matrix.kappa().getInfo())

In [ ]:
# Feature Importance
importance = ee.Dictionary(classifier.explain().get('importance'))
sum_importance = importance.values().reduce(ee.Reducer.sum())
relative_importance = importance.map(lambda k, v: ee.Number(v).multiply(100).divide(sum_importance))

print('Feature Importance (%):', relative_importance.getInfo())

# Optional: Interactive visualization with geemap
# Uncomment if geemap is installed
# Map = geemap.Map()
# Map.centerObject(AOI, 5)
# Map.addLayer(composite, VIS_PARAMS, 'L7 and L8 Composite')
# Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['green', 'red']}, 'Classification')
# Map